# Delight Optimize

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- Creation date : October 25th 2021

- **CC : kernel anaconda3pbspzrail**

**Purpose is to optmize likelihood **

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats
import sys
import os

In [2]:
import sys
from mpi4py import MPI
import numpy as np
from delight.io import *
from delight.utils import *
from delight.photoz_gp import PhotozGP
from delight.photoz_kernels import Photoz_mean_function, Photoz_kernel
from delight.utils_cy import approx_flux_likelihood_cy

In [3]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [4]:
filename_config="../tmp/parametersTest.cfg"
filename_config=="../tmpsim/parametersTest.cfg"

False

In [5]:
# Parse parameters file
params = parseParamFile(filename_config, verbose=True)

2021-10-25 17:39:50,288 ipykernel_launcher.py, delight.io[1583] WARNING Input parameter file:../tmp/parametersTest.cfg
2021-10-25 17:39:50,289 ipykernel_launcher.py, delight.io[1583] WARNING Parameters read:


>  rootDir              ./
>  bands_directory      ./tmp/delight_indata/FILTERS
>  bandNames            lsst_u lsst_g lsst_r lsst_i lsst_z lsst_y
>  numCoefs             15
>  bands_verbose        True
>  bands_debug          True
>  bands_makeplots      False
>  templates_directory  ./tmp/delight_indata/CWW_SEDs
>  lambdaRef            4500.0
>  templates_names      El_B2004a Sbc_B2004a Scd_B2004a SB3_B2004a SB2_B2004a Im_B2004a ssp_25Myr_z008 ssp_5Myr_z008
>  p_t                  [0.27   0.26   0.25   0.069  0.021  0.11   0.0061 0.0079]
>  p_z_t                [0.23 0.39 0.33 0.31 1.1  0.34 1.2  0.14]
>  templates_verbose    False
>  templates_debug      False
>  training_numChunks   1
>  training_paramFile   ./tmp/delight_data/galaxies-gpparams.txt
>  training_catFile     ./tmp/delight_data/galaxies-fluxredshifts.txt
>  training_referenceBand lsst_i
>  training_bandOrder   lsst_u lsst_u_var lsst_g lsst_g_var lsst_r lsst_r_var lsst_i lsst_i_var lsst_z lsst_z_var lsst_y lsst_y_var red

In [6]:
# Read filter coefficients, compute normalization of filters
bandCoefAmplitudes, bandCoefPositions, bandCoefWidths, norms\
    = readBandCoefficients(params)
numBands = bandCoefAmplitudes.shape[0]


In [7]:
redshiftDistGrid, redshiftGrid, redshiftGridGP = createGrids(params)
f_mod = readSEDs(params)
DL = approx_DL()

In [8]:
dir_seds = params['templates_directory']
dir_filters = params['bands_directory']
lambdaRef = params['lambdaRef']
sed_names = params['templates_names']
f_mod_grid = np.zeros((redshiftGrid.size, len(sed_names),
                       len(params['bandNames'])))
for t, sed_name in enumerate(sed_names):
    f_mod_grid[:, t, :] = np.loadtxt(dir_seds + '/' + sed_name +
                                     '_fluxredshiftmod.txt')

In [9]:
numZbins = redshiftDistGrid.size - 1
numZ = redshiftGrid.size
numConfLevels = len(params['confidenceLevels'])
numObjectsTraining = np.sum(1 for line in open(params['training_catFile']))
numObjectsTarget = np.sum(1 for line in open(params['target_catFile']))
print('Number of Training Objects', numObjectsTraining)
print('Number of Target Objects', numObjectsTarget)

Number of Training Objects 3755
Number of Target Objects 10692


/tmp/ipykernel_1583/2030800343.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  numObjectsTraining = np.sum(1 for line in open(params['training_catFile']))
/tmp/ipykernel_1583/2030800343.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  numObjectsTarget = np.sum(1 for line in open(params['target_catFile']))


In [ ]:
for ellPriorSigma in [1.0, 10.0]:
    alpha_C = 1e3
    alpha_L = 1e2
    V_C, V_L = 1.0, 1.0
    gp = PhotozGP(
        f_mod,
        bandCoefAmplitudes, bandCoefPositions, bandCoefWidths,
        params['lines_pos'], params['lines_width'],
        V_C, V_L, alpha_C, alpha_L,
        redshiftGridGP, use_interpolators=True)

    for extraFracFluxError in [1e-2]:
        redshifts = np.zeros((numObjectsTraining, ))
        bestTypes = np.zeros((numObjectsTraining, ), dtype=int)
        ellMLs = np.zeros((numObjectsTraining, ))
        model_mean = np.zeros((numZ, numObjectsTraining, numBands))
        model_covar = np.zeros((numZ, numObjectsTraining, numBands))
        # params['training_extraFracFluxError'] = extraFracFluxError
        params['target_extraFracFluxError'] = extraFracFluxError

        for V_C in [0.1, 1.0]:

            gp.var_C = V_C
            gp.var_L = V_L
            loc = - 1
            trainingDataIter = getDataFromFile(
                params, 0, numObjectsTraining,
                prefix="training_", getXY=True)
            for z, normedRefFlux, bands, fluxes,\
                fluxesVar, bCV, fCV, fvCV, X, Y, Yvar\
                    in trainingDataIter:
                loc += 1
                redshifts[loc] = z
                themod = np.zeros((1, f_mod_grid.shape[1], bands.size))
                for it in range(f_mod_grid.shape[1]):
                    for ib, band in enumerate(bands):
                        themod[0, it, ib] = np.interp(z, redshiftGrid,
                                                      f_mod_grid[:, it, band])
                chi2_grid, theellMLs = scalefree_flux_likelihood(
                    fluxes,
                    fluxesVar,
                    themod,
                    returnChi2=True
                )
                bestTypes[loc] = np.argmin(chi2_grid)
                ellMLs[loc] = theellMLs[0, bestTypes[loc]]
                X[:, 2] = ellMLs[loc]
                gp.setData(X, Y, Yvar, bestTypes[loc])
                model_mean[:, loc, :], model_covar[:, loc, :] =\
                    gp.predictAndInterpolate(redshiftGrid, ell=ellMLs[loc])

            for redshiftSigma in [0.1, 1.0]:

                loc = - 1
                targetDataIter = getDataFromFile(params, 0, numObjectsTarget,
                                                 prefix="target_", getXY=False)

                bias_zmap = np.zeros((redshiftDistGrid.size, ))
                bias_zmean = np.zeros((redshiftDistGrid.size, ))
                confFractions = np.zeros((numConfLevels,
                                          redshiftDistGrid.size))
                binnobj = np.zeros((redshiftDistGrid.size, ))
                bias_nz = np.zeros((redshiftDistGrid.size, ))
                stackedPdfs = np.zeros((redshiftGrid.size,
                                        redshiftDistGrid.size))
                cis = np.zeros((numObjectsTarget, ))
                zmeanBinLocs = np.zeros((numObjectsTarget, ), dtype=int)
                for z, normedRefFlux, bands, fluxes, fluxesVar, bCV, fCV, fvCV\
                        in targetDataIter:
                    loc += 1
                    like_grid = np.zeros((model_mean.shape[0],
                                          model_mean.shape[1]))
                    ell_hat_z = normedRefFlux * 4 * np.pi\
                        * params['fluxLuminosityNorm'] \
                        * (DL(redshiftGrid)**2. * (1+redshiftGrid))
                    ell_hat_z[:] = 1
                    approx_flux_likelihood_cy(
                        like_grid,
                        model_mean.shape[0], model_mean.shape[1], bands.size,
                        fluxes, fluxesVar,
                        model_mean[:, :, bands],
                        V_C*model_covar[:, :, bands],
                        ell_hat_z, (ell_hat_z*ellPriorSigma)**2)
                    like_grid *= np.exp(-0.5*((redshiftGrid[:, None] -
                                               redshifts[None, :]) /
                                              redshiftSigma)**2)
                    pdf = like_grid.sum(axis=1)
                    if pdf.sum() == 0:
                        print("NULL PDF with galaxy", loc)
                    if pdf.sum() > 0:
                        metrics\
                            = computeMetrics(z, redshiftGrid, pdf,
                                             params['confidenceLevels'])
                        ztrue, zmean, zstdzmean, zmap, zstdzmean,\
                            pdfAtZ, cumPdfAtZ = metrics[0:7]
                        confidencelevels = metrics[7:]
                        zmeanBinLoc = -1
                        for i in range(numZbins):
                            if zmean >= redshiftDistGrid[i]\
                                    and zmean < redshiftDistGrid[i+1]:
                                zmeanBinLoc = i
                                bias_zmap[i] += ztrue - zmap
                                bias_zmean[i] += ztrue - zmean
                                binnobj[i] += 1
                                bias_nz[i] += ztrue
                        zmeanBinLocs[loc] = zmeanBinLoc
                        for i in range(numConfLevels):
                            if pdfAtZ >= confidencelevels[i]:
                                confFractions[i, zmeanBinLoc] += 1
                        stackedPdfs[:, zmeanBinLoc]\
                            += pdf / numObjectsTraining
                        ind = pdf >= pdfAtZ
                        pdf /= np.trapz(pdf, x=redshiftGrid)
                        cis[loc] = np.trapz(pdf[ind], x=redshiftGrid[ind])

                confFractions /= binnobj[None, :]
                bias_nz /= binnobj
                for i in range(numZbins):
                    if stackedPdfs[:, i].sum():
                        bias_nz[i] -= np.average(redshiftGrid,
                                                 weights=stackedPdfs[:, i])
                ind = binnobj > 0
                bias_zmap /= binnobj
                bias_zmean /= binnobj
                print("")
                print(' =======================================')
                print("  ellSTD", ellPriorSigma,
                      "fluxError", extraFracFluxError,
                      "V_C", V_C, "zSTD", redshiftSigma)
                cis_pdf, e = np.histogram(cis, 50, range=[0, 1])
                cis_pdfcum = np.cumsum(cis_pdf) / np.sum(cis_pdf)
                print("-------------------------------->>  %.3g"
                      % (np.max(np.abs(np.abs(e[1:] - cis_pdfcum)))))
                print(">>", end="")
                for i in range(numZbins):
                    ind2 = zmeanBinLocs == i
                    if ind2.sum() > 2:
                        cis_pdf, e = np.histogram(cis[ind2], 50, range=[0, 1])
                        cis_pdfcum = np.cumsum(cis_pdf) / np.sum(cis_pdf)
                        print("  %.3g" % (np.max(np.abs(e[1:] - cis_pdfcum))),
                              end=" ")


/tmp/ipykernel_1583/3143773151.py:116: RuntimeWarning: invalid value encountered in true_divide
  confFractions /= binnobj[None, :]
/tmp/ipykernel_1583/3143773151.py:117: RuntimeWarning: invalid value encountered in true_divide
  bias_nz /= binnobj
/tmp/ipykernel_1583/3143773151.py:123: RuntimeWarning: invalid value encountered in true_divide
  bias_zmap /= binnobj
/tmp/ipykernel_1583/3143773151.py:124: RuntimeWarning: invalid value encountered in true_divide
  bias_zmean /= binnobj



  ellSTD 1.0 fluxError 0.01 V_C 0.1 zSTD 0.1
-------------------------------->>  0.565
>>  0.651   0.626   0.544   0.707   0.643   0.507   0.533   0.524   0.522   0.31   0.347   0.401   0.631   0.74   0.48 
  ellSTD 1.0 fluxError 0.01 V_C 0.1 zSTD 1.0
-------------------------------->>  0.553
>>  0.685   0.631   0.555   0.688   0.644   0.505   0.53   0.506   0.505   0.255   0.255   0.299   0.525   0.634   0.94 
  ellSTD 1.0 fluxError 0.01 V_C 1.0 zSTD 0.1
-------------------------------->>  0.565
>>  0.653   0.624   0.551   0.707   0.643   0.507   0.534   0.521   0.528   0.31   0.347   0.401   0.631   0.74   0.647 
  ellSTD 1.0 fluxError 0.01 V_C 1.0 zSTD 1.0
-------------------------------->>  0.554
>>  0.684   0.627   0.557   0.688   0.649   0.503   0.53   0.508   0.501   0.254   0.255   0.294   0.519   0.653   0.94 
  ellSTD 10.0 fluxError 0.01 V_C 0.1 zSTD 0.1
-------------------------------->>  0.566
>>  0.656   0.617   0.537   0.708   0.646   0.511   0.532   0.52   0.523   0.323

In [ ]:
print("")
print(' >>>> mean z bias %.3g'
% np.abs(bias_zmean[ind]).mean(),'mean N(z) bias %.3g' % np.abs(bias_nz[ind]).mean(), ' <<<<')
print(' >>>> max z bias %.3g'% np.abs(bias_zmean[ind]).max(),'max N(z) bias %.3g' % np.abs(bias_nz[ind]).max(), ' <<<<')
print(' > bias_zmap : ','  '.join(['%.3g' % x for x in bias_zmap]))
print(' > z bias : ','  '.join([('%.3g' % x) if np.isfinite(x) else '.' for x in bias_zmean]))
print(' > nzbias : ','  '.join([('%.3g' % x) if np.isfinite(x) else '.' for x in bias_nz]))
print(' --------------------------------')
for i in range(numConfLevels):
    print(' >', params['confidenceLevels'][i],' :: ', '  '.join([('%.3g' % x) if np.isfinite(x) else '.' for x in confFractions[i, :]]))
print(' =======================================')
